### Import dependencies

In [1]:
import os
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display
from huggingface_hub import HfApi, create_repo

### Set variables like HuggingFace token

In [2]:
load_dotenv()  # take environment variables from .env.
HF_TOKEN = os.environ.get("HF_TOKEN")

### Specify source model from HuggingFace

In [3]:
model_id = widgets.Text(
    value='CorticalStack/travel-mistral-7B-16b-base',
    description='Model ID',
    disabled=False
)
display(model_id)

Text(value='CorticalStack/travel-mistral-7B-16b-base', description='Model ID')

### Download the source model

In [4]:
model_name = str(model_id.value).split('/')
if not os.path.isdir(model_name[1]):
    !git clone https://huggingface.co/{model_id.value}

### Set target model name prefix

In [5]:

model_name_prefix = widgets.Text(
    value='travel-mistral-7B',
    description='Model name preix',
    disabled=False
)
display(model_name_prefix)

Text(value='travel-mistral-7B', description='Model name preix')

### Do the GPTQ quantisation

In [6]:
SEQ_LENGTH = 2048
BITS = 4

from optimum.gptq import GPTQQuantizer

# GPTQ quantizer
dataset_id = "wikitext2"
quantizer = GPTQQuantizer(bits=BITS, dataset=dataset_id, model_seqlen=SEQ_LENGTH)
quantizer.quant_method = "gptq"

import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name[1], use_fast=False) # bug with fast tokenizer

# Note I played with setting cuda device and removed low_cpu_mem_usage=True
model = AutoModelForCausalLM.from_pretrained(model_name[1], device_map="cuda:0", torch_dtype=torch.float16) # we load the model in fp16 on purpose

# quantize the model
quantized_model = quantizer.quantize_model(model, tokenizer)

# save the quantize model to disk
save_folder = model_name_prefix.value + "-GPTQ"
quantized_model.save_pretrained(save_folder, safe_serialization=True)

# load fresh, fast tokenizer and save it to disk
tokenizer = AutoTokenizer.from_pretrained(model_name[1]).save_pretrained(save_folder)

# save quantize_config.json for TGI
with open(os.path.join(save_folder, "quantize_config.json"), "w", encoding="utf-8") as f:
  quantizer.disable_exllama = False
  json.dump(quantizer.to_dict(), f, indent=2)

CUDA extension not installed.
CUDA extension not installed.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2763962 > 32768). Running this sequence through the model will result in indexing errors


Quantizing model.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Using Exllamav2 backend will reorder the weights offline, thus you will not be able to save the model with the right weights.Setting `disable_exllama=True`. You should only use Exllamav2 backend for inference. 


### Push to HuggingFace Hub

In [7]:
api = HfApi()

model_id = f"CorticalStack/{save_folder}"
api.create_repo(model_id, exist_ok=True, repo_type="model", token=HF_TOKEN)

# Upload gptq files
api.upload_folder(
    folder_path=save_folder,
    repo_id=model_id,
    token=HF_TOKEN
)

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/CorticalStack/travel-mistral-7B-GPTQ/commit/e276ea987146cfb2af9fe76427275e601fa62bd7', commit_message='Upload folder using huggingface_hub', commit_description='', oid='e276ea987146cfb2af9fe76427275e601fa62bd7', pr_url=None, pr_revision=None, pr_num=None)